# Week 13 — Open Games: Attention Economy (Co‑Play Demo)
We model a tiny attention economy as an **open game**.
The forward pass **plays** (allocates attention), while the backward pass **co‑plays** (propagates value).
This is a lightweight, code‑friendly illustration of open‑game notation.


**Open game notation (informal)**
- An open game has two interfaces: forward (play) and backward (co‑play).
- Think of it as a box with arrows:  \n  **play:** inputs → outputs (resources / actions)  \n  **coplay:** outputs' value → inputs' value (incentives / prices)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)


## 1) A tiny open‑game interface
We define a minimal structure with `play` and `coplay`.


In [ ]:
class OpenGame:
    def __init__(self, play, coplay, name='game'):
        self.play = play
        self.coplay = coplay
        self.name = name

    def __rshift__(self, other):
        # Sequential composition: g >> h
        def play(x):
            y = self.play(x)
            z = other.play(y)
            return z

        def coplay(v):
            # backward value through other, then self
            v_y = other.coplay(v)
            v_x = self.coplay(v_y)
            return v_x

        return OpenGame(play, coplay, name=f'{self.name}>>{other.name}')


## 2) Producer → Transporter → Consumers
We have one producer (unconscious), one transporter (pathway), and two consumers (memory slots).
The producer emits a resource vector, the transporter allocates it, and consumers yield rewards.


In [ ]:
# Producer: emits a fixed resource budget
def producer_play(_):
    return np.array([1.0, 1.0])  # two channels

def producer_coplay(value_vec):
    # producer receives the aggregate value signal
    return np.sum(value_vec)

producer = OpenGame(producer_play, producer_coplay, name='producer')

# Transporter: allocates resources via a softmax gate
def make_transporter(alpha=1.0):
    def play(res):
        # res: (2,) available resources
        gate = np.exp(alpha * np.array([0.3, 0.7]))
        gate = gate / gate.sum()
        alloc = res * gate
        return alloc
    def coplay(values):
        # values: marginal value of each channel
        return values  # pass-through price signal
    return OpenGame(play, coplay, name='transporter')

transporter = make_transporter(alpha=2.0)

# Consumers: convert allocation to reward (utility)
def consumers_play(alloc):
    # concave utility: diminishing returns
    u = np.sqrt(alloc + 1e-9)
    return u

def consumers_coplay(u):
    # marginal value (derivative of sqrt)
    return 0.5 / np.sqrt(u + 1e-9)

consumers = OpenGame(consumers_play, consumers_coplay, name='consumers')


## 3) Compose the open games
We compose **producer → transporter → consumers** and inspect play + coplay.


In [ ]:
economy = producer >> transporter >> consumers

alloc = transporter.play(producer.play(None))
utility = consumers.play(alloc)
value_signal = consumers.coplay(utility)
back_to_producer = transporter.coplay(value_signal)
producer_value = producer.coplay(back_to_producer)

print('Allocation:', alloc)
print('Utility:', utility)
print('Value signal (coplay):', value_signal)
print('Producer aggregated value:', producer_value)


## 4) Vary transporter gate (attention competition)
We sweep the gate parameter and see how allocation and value change.


In [ ]:
alphas = np.linspace(0.1, 4.0, 30)
alloc_hist = []
value_hist = []

for a in alphas:
    transporter = make_transporter(alpha=a)
    alloc = transporter.play(producer.play(None))
    util = consumers.play(alloc)
    val = consumers.coplay(util)
    alloc_hist.append(alloc)
    value_hist.append(val)

alloc_hist = np.array(alloc_hist)
value_hist = np.array(value_hist)

plt.figure(figsize=(6,4))
plt.plot(alphas, alloc_hist[:,0], label='alloc channel 0')
plt.plot(alphas, alloc_hist[:,1], label='alloc channel 1')
plt.xlabel('gate sharpness (alpha)')
plt.ylabel('allocated attention')
plt.title('Play: allocation vs gate')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
plt.plot(alphas, value_hist[:,0], label='value channel 0')
plt.plot(alphas, value_hist[:,1], label='value channel 1')
plt.xlabel('gate sharpness (alpha)')
plt.ylabel('marginal value (coplay)')
plt.title('Co‑play: value signals vs gate')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


## Takeaway
- **Play**: how attention/resources flow forward.
- **Co‑play**: how value/credit flows backward.
- This simple compositional setup mirrors open‑game intuition without heavy machinery.
